In [2]:
import numpy as np
from scipy.stats import norm

# Generate synthetic data for demonstration
np.random.seed(42)
data = np.concatenate([np.random.normal(0, 1, 200), np.random.normal(5, 1, 200)])

# Initialize parameters
num_components = 2  # Number of Gaussian components
means = np.random.choice(data, num_components)
variances = np.random.random(num_components)
weights = np.ones(num_components) / num_components

def e_step(data, means, variances, weights):
    """E-step: calculate responsibilities."""
    responsibilities = np.zeros((len(data), num_components))
    for k in range(num_components):
        responsibilities[:, k] = weights[k] * norm.pdf(data, means[k], np.sqrt(variances[k]))
    responsibilities /= responsibilities.sum(axis=1, keepdims=True)
    return responsibilities

def m_step(data, responsibilities):
    """M-step: update parameters."""
    n_k = responsibilities.sum(axis=0)
    weights = n_k / len(data)
    means = (responsibilities.T @ data) / n_k
    # The following line has been modified
    variances = (np.sum(responsibilities * (data[:, np.newaxis] - means)**2, axis=0)) / n_k
    return weights, means, variances

def log_likelihood(data, means, variances, weights):
    """Compute the log likelihood of the data given the model parameters."""
    likelihood = np.zeros(len(data))
    for k in range(num_components):
        likelihood += weights[k] * norm.pdf(data, means[k], np.sqrt(variances[k]))
    return np.sum(np.log(likelihood))

# EM algorithm
max_iters = 100
tolerance = 1e-6
log_likelihoods = []

for i in range(max_iters):
    # E-step
    responsibilities = e_step(data, means, variances, weights)

    # M-step
    weights, means, variances = m_step(data, responsibilities)

    # Compute log-likelihood
    log_likelihood_value = log_likelihood(data, means, variances, weights)
    log_likelihoods.append(log_likelihood_value)

    # Check for convergence
    if i > 0 and abs(log_likelihoods[-1] - log_likelihoods[-2]) < tolerance:
        print(f"Converged at iteration {i}.")
        break

# Results
print("Final means:", means)
print("Final variances:", variances)
print("Final weights:", weights)

Converged at iteration 9.
Final means: [-0.05233444  5.07979589]
Final variances: [0.83075106 0.97249109]
Final weights: [0.49828183 0.50171817]
